In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastai.callback.wandb import WandbCallback
from fastxtend.vision.all import EMACallback

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_41'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV7Flip'
    sn_train = False
    
    model_name = 'RNA_ModelV25U'
    model_kwargs = dict(dim=192 * 2,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 6)

   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    wandb = True
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

In [4]:
def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [5]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
bpp_df.drop_duplicates(subset = 'sequence_id', inplace=True)
ss = pd.read_parquet(CFG.pathss)[["sequence_id", "ss_full"]]
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df = pd.merge(df, ss,  on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train', mask_only=True,sn_train=CFG.sn_train)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
del bpp_df
del ss
del split
del df
gc.collect()


16

In [7]:
if CFG.wandb:
    wandb.init(
        project="rna",
        entity="drhb",
        name=CFG.out,
        config=class_to_dict(CFG),
    )
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                     WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)
wandb.finish()

wandb: Currently logged in as: drhb. Use `wandb login --relogin` to force relogin


Could not gather input dimensions


epoch,train_loss,valid_loss,mae,time
0,0.126759,0.151922,0.156273,59:08
1,0.122071,0.138068,0.145970,58:53
2,0.121573,0.130668,0.140905,58:43
3,0.119079,0.128600,0.138765,57:49
4,0.117155,0.127592,0.137698,57:43
5,0.117803,0.126996,0.137081,57:42
6,0.117374,0.126439,0.136490,57:46
7,0.116009,0.126072,0.136115,57:47


Better model found at epoch 0 with mae value: 0.15627274248144918.
Better model found at epoch 1 with mae value: 0.14596982808892117.
Better model found at epoch 2 with mae value: 0.1409053038677537.
Better model found at epoch 3 with mae value: 0.13876476484722258.
Better model found at epoch 4 with mae value: 0.1376979120850007.
Better model found at epoch 5 with mae value: 0.1370806182983293.
Better model found at epoch 6 with mae value: 0.1364903881645784.
Better model found at epoch 7 with mae value: 0.13611536616576117.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 │   │   │   │   │    EMACallback(),                                                         │
│   16 │   │   │   │   │   SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],             │
│   17 │   │   │   │   metrics=[MAE()]).to_fp16()                                                  │
│ ❱ 18 learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)           │
│   19 wandb.finish()                                                                              │
│   20                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/callback/schedule.py:119 in fit_one_cycle         │
│                                                                                                  │
│   116 │   lr_max = np.array([h['lr'] for h in self.opt.hypers])                                  │
│   117 │   scheds = {'lr': combined_cos(pct_start, lr_max/div, lr_max, lr_max/div_final),         │
│   118 │   │   │     'mom': combined_cos(pct_start, *(self.moms if moms is None else moms))}      │
│ ❱ 119 │   self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, sta   │
│   120                                                                                            │
│   121 # %% ../../nbs/14_callback.schedule.ipynb 50                                               │
│   122 @patch                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/learner.py:264 in fit                             │
│                                                                                                  │
│   261 │   │   │   if wd is not None: self.opt.set_hypers(wd=wd)                                  │
│   262 │   │   │   self.opt.set_hypers(lr=self.lr if lr is None else lr)                          │
│   263 │   │   │   self.n_epoch = n_epoch                                                         │
│ ❱ 264 │   │   │   self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup   │
│   265 │                                                                                          │
│   266 │   def _end_cleanup(self): self.dl,self.xb,self.yb,self.pred,self.loss = None,(None,),(   │
│   267 │   def __enter__(self): self(_before_epoch); return self                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/learner.py:199 in _with_events                    │
│                                                                                                  │
│   196 │   │   self.xb,self.yb = b[:i],b[i:]                                                      │
│   197 │                                                                                          │
│   198 │   def _with_events(self, f, event_type, ex, final=noop):                                 │
│ ❱ 199 │   │   try: self(f'before_{event_type}');  f()                                            │
│   200 │   │   except ex: self(f'after_cancel_{event_type}')                                      │
│   201 │   │   self(f'after_{event_type}');  final()                                              │
│   202                                                                                            │
│                                                            

In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim, graph_layers=CFG.graph_layers).cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")

In [14]:
ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train[595200:].reset_index(drop=True), mode='train',sn_train=CFG.sn_train)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, batch_size=64, 
            shuffle=False, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)

In [15]:
from tqdm import tqdm
for x, y in tqdm(dl_train):
    pass

 84%|███████████████████████████████████████████████████████████████████████████▊              | 4719/5603 [01:45<00:19, 44.82it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 from tqdm import tqdm                                                                        │
│ ❱ 2 for x, y in tqdm(dl_train):                                                                  │
│   3 │   pass                                                                                     │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/tqdm/std.py:1178 in __iter__                             │
│                                                                                                  │
│   1175 │   │   time = self._time                                                                 │
│   1176 │   │                                                                                     │
│   1177 │   │   try:                                                                              │
│ ❱ 1178 │   │   │   for obj in iterable:                                                          │
│   1179 │   │   │   │   yield obj                                                                 │
│   1180 │   │   │   │   # Update and possibly print the progressbar.                              │
│   1181 │   │   │   │   # Note: does not call self.update(1) for speed optimisation.              │
│                                                                                                  │
│ /opt/slh/rna/exp/../rnacomp/dataset.py:84 in __iter__                                            │
│                                                                                                  │
│     81 │   │   return len(self.dataloader)                                                       │
│     82 │                                                                                         │
│     83 │   def __iter__(self):                                                                   │
│ ❱   84 │   │   for batch in self.dataloader:                                                     │
│     85 │   │   │   yield tuple(dict_to(x, self.device) for x in batch)                           │
│     86                                                                                           │
│     87 # %% ../nbs/00_dataset.ipynb 6                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:1325 in _next_data        │
│                                                            

In [26]:
for i in tqdm(range(251001, len(ds_train))):
    ds_train[i]
    

 47%|████████████████████████████████████████▊                                             | 51031/107572 [02:54<03:13, 292.80it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for i in tqdm(range(251001, len(ds_train))):                                                 │
│ ❱ 2 │   ds_train[i]                                                                              │
│   3                                                                                              │
│   4                                                                                              │
│                                                                                                  │
│ /opt/slh/rna/exp/../rnacomp/dataset.py:2140 in __getitem__                                       │
│                                                                                                  │
│   2137 │   │   sn = torch.FloatTensor([self.sn_2A3[idx], self.sn_DMS[idx]])                      │
│   2138 │   │                                                                                     │
│   2139 │   │   ss_adj = torch.tensor(dot_to_adjacencyv0(self.ss[idx], L)).int()                  │
│ ❱ 2140 │   │   bpp = generate_base_pair_matrixv1(self.bpp[idx], L).float()                       │
│   2141 │   │                                                                                     │
│   2142 │   │                                                                                     │
│   2143 │   │   bpp_extra = [                                                                     │
│                                                                                                  │
│ /opt/slh/rna/exp/../rnacomp/dataset.py:698 in generate_base_pair_matrixv1                        │
│                                                                                                  │
│    695 │   # Find the largest position in the 'pos1' column                                      │
│    696 │   largest_position = data["pos1"].max()                                                 │
│    697 │                                                                                         │
│ ❱  698 │   ids = torch.from_numpy(data[["pos1", "pos2"]].values.astype(int))                     │
│    699 │   matrix = torch.zeros((L, L))                                                          │
│    700 │   matrix[ids[:, 0] - 1, ids[:, 1] - 1] = torch.from_numpy(data["prob"].values).float()  │
│    701 │   matrix[ids[:, 1] - 1, ids[:, 0] - 1] = torch.from_numpy(data["prob"].values).float()  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: invalid literal for int() with base 10: '123$4.8843e-03'

In [27]:
%debug

> /opt/slh/rna/rnacomp/dataset.py(698)generate_base_pair_matrixv1()
    696     largest_position = data["pos1"].max()
    697 
--> 698     ids = torch.from_numpy(data[["pos1", "pos2"]].values.astype(int))
    699     matrix = torch.zeros((L, L))
    700     matrix[ids[:, 0] - 1, ids[:, 1] - 1] = torch.from_numpy(data["prob"].values).float()



ipdb>  fn


*** NameError: name 'fn' is not defined


ipdb>  name


*** NameError: name 'name' is not defined


ipdb>  file_path


Path('../data/Ribonanza_bpp_files/extra_data/5/9/c/908c3030eb81.txt')


ipdb>  exit


In [25]:
len(ds_train)

358573